In [9]:
from multiprocessing_scripts import sentiment_analysis, emoji_count, emoji_index
import pandas as pd
import numpy as np
import sqlite3
import emoji

### Removing NAs and Duplicates REVIEW

In [2]:
con_rev = sqlite3.connect('../database/review.db')
df_rev = pd.read_sql_query("SELECT * from review", con_rev)

In [ ]:
df_rev['note_length'] = df_rev.apply(lambda row: len(str(row.note).split()), axis=1)

In [11]:
print(len(df_rev))
df_rev.dropna(how='any', inplace=True)
df_rev.note.dropna(how='any', inplace=True)
# Remove ratings with empty notes
df_rev = df_rev[df_rev.note_length > 0]
df_rev.id.drop_duplicates(inplace=True)
print(len(df_rev))

8518578
8518578


In [7]:
df_rev.note = df_rev.note.str.lower()
df_rev.note = df_rev.note.str.replace('vivino', '')
df_rev.note = df_rev.note.str.replace('\n', '')

In [8]:
df_rev.sort_values(by='id', inplace=True)
if 'index' in df_rev.columns:
    df_rev.drop(columns='index', inplace=True)
df_rev.to_sql('review', con=con_rev, if_exists='replace')

8518578

### Transform Emojis to CLDR short name

In [3]:
con_rev = sqlite3.connect('../database/review.db')
df_rev = pd.read_sql_query("SELECT * from review", con_rev)

In [5]:
emoji_list = list(emoji.EMOJI_DATA.keys())
emoji_list = '|'.join(emoji_list)
# *-symbol causes an error with regex -> remove
emoji_list = emoji_list.replace('*', '')

In [6]:
emoji_indexes = emoji_index.process(df=df_rev, emoji_list=emoji_list)

In [3]:
def replace_emojis(df: pd.DataFrame, indexes):
    for index in indexes:
        note: str = df.loc[index, 'note']
        emojis = emoji.distinct_emoji_list(note)
        for var_emoji in emojis:
            emoji_name = emoji.demojize(var_emoji).replace(':', '').replace('_', ' ')
            note = note.replace(var_emoji, emoji_name + ' ')
        df.loc[index, 'note'] = note
    return df

In [8]:
df_rev_no_emoji = replace_emojis(df=df_rev, indexes=emoji_indexes)

In [63]:
con_rev_no_emojis = sqlite3.connect('../database/review_no_emojis.db')
df_rev_no_emoji.sort_values(by='id', inplace=True)
if 'index' in df_rev_no_emoji.columns:
    df_rev_no_emoji.drop(columns='index', inplace=True)
df_rev_no_emoji.to_sql('review_no_emojis', con=con_rev_no_emojis, if_exists='replace')

8518578

### Spelling correction REVIEW-NO-EMOJIS

In [2]:
con_rev_sentiment = sqlite3.connect('../database/review_en_sentiment.db')
df_rev_sentiment = pd.read_sql_query("SELECT id, note FROM review_en_sentiment", con_rev_sentiment)
con_rev_sentiment.close()
df_rev_sentiment.head(n=1)

,id,note
0,3975,excellent full bodied wine


In [ ]:
from neuspell import BertChecker

bc = BertChecker()
bc.from_pretrained()

In [8]:
note_sample = df_rev_sentiment.note.sample(n=20)

for note in note_sample:
    print(
        'Original: ' + note + '\n' +
        'Corrected: ' + bc.correct(note) + '\n'
    )

Original: my 1st white burgundy.  i liked it but nothing too special.  it reminded me of a lighter sauvignon blanc.
Corrected: my 1st white burgundy . i liked it but nothing too special . it reminded me of a lighter sauvignon blanc .

Original: (4.0)100% romorantin (probably cross of pn&gouais, exclusively in cour-cheverny aoc). biodynamic.med intense nose, very fresh and lively, mix of acacia, green vegetables, citrus fruits and minerals.light+ bodied, dry, very mineral, vigorous, flavors of silex, green herbs, some lime and honey.great finish, long, highly zippy with grapefruit zest.reportedly close to chablis, but that one with more fruity impact. 
Corrected: ( 4 . 0 ) 100 % romorantin ( probably cross of pink & gouais , exclusively in four - cheverny acid ) . biodynamic . med intense nose , very fresh and lively , mix of acacia , green vegetables , citrus fruits and minerals . light + bodied , dry , very mineral , vigorous , flavors of sales , green herbs , some lime and honey . gr

In [4]:
from tqdm import tqdm
from datetime import datetime


df = df_rev_sentiment
start = datetime.now()

for i in tqdm(df.index):
    df.loc[i, 'note'] = bc.correct(df.loc[i, 'note']).lower()

print(datetime.now()- start)

con_rev_sentiment_sc = sqlite3.connect('../database/review_en_sentiment_sc.db')
df.to_sql('review_en_sentiment_sc', con=con_rev_sentiment_sc, if_exists='replace')

loading vocab from path:/Users/leonbecker/neuspell/neuspell/../data/checkpoints/subwordbert-probwordnoise/vocab.pkl
initializing model


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Number of parameters in the model: 185211810
Loading model params from checkpoint dir: /Users/leonbecker/neuspell/neuspell/../data/checkpoints/subwordbert-probwordnoise


100%|██████████| 3976851/3976851 [66:48:20<00:00, 16.54it/s]    


2 days, 18:48:20.852025


3976851

In [6]:
con_rev_sentiment = sqlite3.connect('../database/review_en_sentiment.db')
df_rev_sentiment = pd.read_sql_query("SELECT * FROM review_en_sentiment", con_rev_sentiment)
con_rev_sentiment.close()
df_rev_sentiment.head(n=1)

,index,id,wine_id,rating,note,created_at,likes_count,comments_count,scan_image_path,user_id,note_length,len_code,sentiment
0,0,3975,1101258,4.0,excellent full bodied wine,2011-07-19T21:21:22.000Z,2.0,1.0,images.vivino.com/labels/1309631550_2382.jpg,32547,4,en,0.938549


In [7]:
df_sc = pd.merge(df_rev_sentiment, df, on='id')
df_sc.head()

,index,id,wine_id,rating,note_x,created_at,likes_count,comments_count,scan_image_path,user_id,note_length,len_code,sentiment,note_y
0,0,3975,1101258,4.0,excellent full bodied wine,2011-07-19T21:21:22.000Z,2.0,1.0,images.vivino.com/labels/1309631550_2382.jpg,32547,4,en,0.938549,excellent full bodied wine
1,1,4202,76378,5.0,the real price at phillipson is around 1299 dk...,2011-08-06T20:52:53.000Z,0.0,1.0,images.vivino.com/labels/1311959128_1819.jpg,48500,39,en,0.576068,the real price at phillipson is around 1299 dk...
2,2,4261,1231210,5.0,"clean, crisp and full of nuance and elegance, ...",2011-08-11T08:06:39.000Z,3.0,1.0,images.vivino.com/labels/1312622458_6592.jpg,53601,17,en,0.982237,"clean , crisp and full of nuance and elegance ..."
3,3,4264,1178663,4.0,"crisp and clean champagne, a certain fave amon...",2011-08-11T09:28:07.000Z,25.0,2.0,images.vivino.com/labels/1312966707_6129.jpg,53601,22,en,0.576195,"crisp and clean champagne , a certain face amo..."
4,4,4354,17998,4.0,it is not a champagne type wine! it is a red w...,2011-08-12T16:53:19.000Z,0.0,1.0,images.vivino.com/labels/1311842730_8268.jpg,47873,12,en,0.736812,it is not a champagne type wine ! it is a red ...


In [14]:
df_sc.drop(columns=['note_x'], inplace=True)
df_sc.rename(columns={'note_y': 'note'}, inplace=True)
df_sc.head()

,index,id,wine_id,rating,created_at,likes_count,comments_count,scan_image_path,user_id,note_length,len_code,sentiment,note
0,0,3975,1101258,4.0,2011-07-19T21:21:22.000Z,2.0,1.0,images.vivino.com/labels/1309631550_2382.jpg,32547,4,en,0.938549,excellent full bodied wine
1,1,4202,76378,5.0,2011-08-06T20:52:53.000Z,0.0,1.0,images.vivino.com/labels/1311959128_1819.jpg,48500,39,en,0.576068,the real price at phillipson is around 1299 dk...
2,2,4261,1231210,5.0,2011-08-11T08:06:39.000Z,3.0,1.0,images.vivino.com/labels/1312622458_6592.jpg,53601,17,en,0.982237,"clean , crisp and full of nuance and elegance ..."
3,3,4264,1178663,4.0,2011-08-11T09:28:07.000Z,25.0,2.0,images.vivino.com/labels/1312966707_6129.jpg,53601,22,en,0.576195,"crisp and clean champagne , a certain face amo..."
4,4,4354,17998,4.0,2011-08-12T16:53:19.000Z,0.0,1.0,images.vivino.com/labels/1311842730_8268.jpg,47873,12,en,0.736812,it is not a champagne type wine ! it is a red ...


In [25]:
con_rev_sentiment_sc = sqlite3.connect('../database/review_en_sentiment_sc.db')
if 'index' in df_sc.columns:
    df_sc.drop(columns='index', inplace=True)
df_sc.drop_duplicates(subset='id', inplace=True)
df_sc.to_sql('review_en_sentiment_sc', con=con_rev_sentiment_sc, if_exists='replace')

3976746

### Sentiment analysis

In [ ]:
con_rev = sqlite3.connect('../database/review_en.db')
df_rev = pd.read_sql_query("SELECT id, note from review_en", con_rev)
results = sentiment_analysis.process(df_rev)
df_results = pd.concat(results)
del df_rev
con_reviews = sqlite3.connect('../database/review_en.db')
df_reviews = pd.read_sql_query("SELECT * from review_en", con_reviews)
df_reviews.drop(columns=['index'], inplace=True)
df_final = pd.merge(df_reviews, df_results, on='id')

con_rev_en_sent = sqlite3.connect('../database/review_en_sentiment.db')
cur_rev_en_sent = con_rev_en_sent.cursor()
cur_rev_en_sent.execute('CREATE TABLE IF NOT EXISTS review_en_sentiment('
                        'id, wine_id, rating, note, created_at, likes_count, comments_count, scan_image_path, user_id, note_length, len_code, sentiment)')
df_final.to_sql('review_en_sentiment', con=con_rev_en_sent, if_exists='replace')

In [4]:
con_rev_en = sqlite3.connect('../database/review_en.db')
df_rev_en = pd.read_sql_query("SELECT note from review_en", con_rev_en)
df_rev_en.head()

,note
0,excellent full bodied wine
1,the real price at phillipson is around 1299 dk...
2,"clean, crisp and full of nuance and elegance, ..."
3,"crisp and clean champagne, a certain fave amon..."
4,it is not a champagne type wine! it is a red w...


### Emoji count REVIEW

In [3]:
emoji_list = list(emoji.EMOJI_DATA.keys())
emoji_list = '|'.join(emoji_list)
# *-symbol causes an error with regex -> remove
emoji_list = emoji_list.replace('*', '')
emoji_count = emoji_count.process(df_rev, emoji_list)

In [6]:
(emoji_count / len(df_rev)) * 100

4.602235255696431

### Emoji count REVIEW_EN

In [5]:
emoji_list = list(emoji.EMOJI_DATA.keys())
emoji_list = '|'.join(emoji_list)
# *-symbol causes an error with regex -> remove
emoji_list = emoji_list.replace('*', '')
emoji_count_en = emoji_count.process(
    df=df_rev_en,
    emoji_list=emoji_list
)
(emoji_count_en / len(df_rev_en)) * 100

4.18031568748694

### Training, Test & Validation Dataset

In [2]:
from sklearn.model_selection import train_test_split

con_rev_en_sent = sqlite3.connect('../database/review_en_sentiment.db')
df_rev_en_sent = pd.read_sql_query("SELECT * from review_en_sentiment", con_rev_en_sent)
df_rev_en_sent.head(n=2)

,index,id,wine_id,rating,note,created_at,likes_count,comments_count,scan_image_path,user_id,note_length,len_code,sentiment
0,0,3975,1101258,4.0,excellent full bodied wine,2011-07-19T21:21:22.000Z,2.0,1.0,images.vivino.com/labels/1309631550_2382.jpg,32547,4,en,0.938549
1,1,4202,76378,5.0,the real price at phillipson is around 1299 dk...,2011-08-06T20:52:53.000Z,0.0,1.0,images.vivino.com/labels/1311959128_1819.jpg,48500,39,en,0.576068


In [18]:
train, validation, test = \
              np.split(df_rev_en_sent.sample(frac=1, random_state=26),
                                [int(0.8*len(df_rev_en_sent)), int(0.9*len(df_rev_en_sent))])
print(
    'train ratio: ' +  str(len(train)/len(df_rev_en_sent))      + '\n' +
    'val ratio: ' +    str(len(validation)/len(df_rev_en_sent))   + '\n' +
    'test ratio: ' +   str(len(test)/len(df_rev_en_sent))       + '\n'
)

train ratio: 0.7999997988358125
val ratio: 0.09999997485447656
test ratio: 0.10000022630971088



In [19]:
con_rev_en_train = sqlite3.connect('../finetuing_datasets/review_en_train.db')
if 'index' in train.columns:
    train.drop(columns='index', inplace=True)
train.to_sql('review_en_train', con=con_rev_en_train, if_exists='replace')
con_rev_en_train.close()

con_rev_en_test = sqlite3.connect('../finetuing_datasets/review_en_test.db')
if 'index' in test.columns:
    test.drop(columns='index', inplace=True)
test.to_sql('review_en_test', con=con_rev_en_test, if_exists='replace')
con_rev_en_test.close()

con_rev_en_val = sqlite3.connect('../finetuing_datasets/review_en_validation.db')
if 'index' in validation.columns:
    validation.drop(columns='index', inplace=True)
validation.to_sql('review_en_validation', con=con_rev_en_val, if_exists='replace')
con_rev_en_val.close()